In [25]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [26]:
# Get images
image = img_to_array(load_img('C:/Study/Semester2/Machine Learning/smaller2/boy1.jpg'))
image = np.array(image, dtype=float)

In [27]:
print('shape:', image.shape,'min: ', np.min(image),'max: ', np.max(image))

shape: (750, 600, 3) min:  0.0 max:  255.0


In [28]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
X = X.reshape(1, 750, 600, 1)
Y = Y.reshape(1, 750, 600, 2)

In [29]:
print('shape:', X.shape,'min: ', np.min(X),'max: ', np.max(X)) #BW is in the scale of 0 to 100
print('shape:', Y.shape,'min: ', np.min(Y),'max: ', np.max(Y)) #Color is in the scale of 0 to 1 

shape: (1, 750, 600, 1) min:  0.1364043301802944 max:  100.0
shape: (1, 750, 600, 2) min:  -0.15990399927543422 max:  0.28320231633933557


In [53]:

# Building the neural network
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides = 2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides = 2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides = 2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [54]:
# Finish model
model.compile(optimizer='rmsprop',loss='mse')

In [52]:
model.fit(x=X, 
    y=Y,
    batch_size=1,
    epochs=1000)

Epoch 1/1000


ValueError: in user code:

    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "C:\Users\midyr\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:

In [49]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_69 (Conv2D)          (None, None, None, 8)     80        
                                                                 
 conv2d_70 (Conv2D)          (None, None, None, 8)     584       
                                                                 
 conv2d_71 (Conv2D)          (None, None, None, 16)    1168      
                                                                 
 conv2d_72 (Conv2D)          (None, None, None, 16)    2320      
                                                                 
 conv2d_73 (Conv2D)          (None, None, None, 32)    4640      
                                                                 
 conv2d_74 (Conv2D)          (None, None, None, 32)    9248      
                                                                 
 up_sampling2d_22 (UpSamplin  (None, None, None, 32)  

In [ ]:
print(model.evaluate(X, Y, batch_size=1))
output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))